# Blockchain

Using JSON (javascript object notation) to trasnport data because lightweight, often used for server-> webpage data movement, 'self-describing'.

- Data in name value pairs
- Data seperated by commas
- Curly brackets hold objects
- Square brackets hold arrays

eg:
{
"employees":[

    {"firstName":"John", "lastName":"Doe"},
    {"firstName":"Anna", "lastName":"Smith"},
    {"firstName":"Peter", "lastName":"Jones"}
    
]
}

Obj employees is an array with 3 objects. Each obj is a record of a person witha first and last name. 

For the __HASH__, look at various crypto methods. For storing passwords and usernames, also look at salt (crypto). Each transaction in block is usually hashed, resulting in a hash or __merkel tree__.

__object.--dict--__ is a special attribute that returns the obj as a dictionary (example below).

Hash's can be __stored__ in a python __list__. __Dependency__ is required to prevent invalidation. Taking an old block and replacing it with a new one can occur, so something must be done. This can be achieved by using the previous block's hash in the current one. The first block is called the __genesis block__. It can be uniquelly created by logic ('randomly') or manually.If content of any block prev changes, _prev-hash_ field will have a __mismatch__, thus the entire chain must be recomputed. 

It is possible to recompute, thus add a __constraint__ of some kind. New Block class attribute called __nonce__.

Blocks can be added to the chain if they are __untampered__ and the __transaction order__ is preserved (prev_hash in block to be added points to hash of last block in chain).

#### Also See: 
Merkel tree  
HashCash for proof of work

In [14]:
class MyClass(object):
    class_var = 1

    def __init__(self, i_vaw, b_var):
        self.i_var = i_vaw
        self.b_var = b_var

foo = MyClass(2, 4)
bar = MyClass(3, 6)

print (MyClass.__dict__)
print (foo.__dict__)
print (bar.__dict__)


{'__module__': '__main__', 'class_var': 1, '__init__': <function MyClass.__init__ at 0x7f7c0c2c1950>, '__dict__': <attribute '__dict__' of 'MyClass' objects>, '__weakref__': <attribute '__weakref__' of 'MyClass' objects>, '__doc__': None}
{'i_var': 2, 'b_var': 4}
{'i_var': 3, 'b_var': 6}


In [1]:
from hashlib import sha256
import json
import time
import pdb   # pdb.set_trace()

class Block:
    def __init__(self, index, transac, timestamp, prev_hash):
        """ Block constructor
            :param index: unique  block ID
            :param transac: list of transactions
            :param timestamp: Time generation of block
            :param previous_hash: Previous block's hash of which this is a part off """
        self.index = index
        self.transac = transac
        self.timestamp = timestamp
        self.prev_hash = prev_hash

    def hasher(self):
        """ Returns hash of block after convertnig
            to JSON string """
        block_json_str = json.dumps(self.__dict__, sort_keys=True)
        return sha256(block_json_str.encode()).hexdigest()
    
class Blockchain:
    def __init__(self):
        """ Blockchain constructor """
        self.chain = []
        self.genesis_block()
        
    def genesis_block(self):
        """ Generates the index 0 
        block and appends it to chain """
        gb = Block(0, [], time.time(), "0")            # Block obj
        gb.hash = gb.hasher()                          # Created Attribute
        self.chain.append(gb)
        
    def last_block(self):
        """ Retrieves last block.
            Will always have at least one (genesis) """
        return self.chain[-1]
    
    def proof_of_work(self, block):
        """ Tries diff vals of nonce to get 
            hash that satisfies difficulty """
        block.nonce = 0 #created attribute
        
    def add_block(self, block, proof):
        """ Func which adds block to chain after verification
            * Checks if proof is valid.
            * Checks if prev_hash of block matches latest block. """
        previous_hash = self.last_block.hash
        
        if previous_hash != block.prev_hash:
            return False
        if not Blockchain.is_valid_proof(block, proof):
            return False
        
        block.hash = proof
        self.chain.append(block)
        return True

    def is_valid_proof(self, block, block_hash):
        """ Check is block_hash is valid & satisfies difficulty """
        return (block_hash.startwith('0'*Blockchain.difficulty) and block_hash == block.compute_hash())

q = Blockchain()


In [7]:
data = b"Hello World"
sha256(data).hexdigest()

'a591a6d40bf420404a011733cfb7b190d62c65bf0bcda32b57b277d9ad9f146e'

In [8]:
data = b"Hello World"
sha256(data).hexdigest()

'a591a6d40bf420404a011733cfb7b190d62c65bf0bcda32b57b277d9ad9f146e'

In [9]:
data = b"Hello1 World"
sha256(data).hexdigest()

'b67ac8c5bbbe3e844cfda6eb8c93f224683d18f4b6ad9a8cb2877991165103e2'

In [16]:
time.time()

NameError: name 'time' is not defined